In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy.typing as npt
import matplotlib.pyplot as plt

#TODO
- fazer plot da legenda ficar sempre no canto superior esquerdo
- por titulo na legenda dizendo qual variṕavel esta avriando
- pegar chave e valor de valores fixos de forma mais elegante

In [2]:
def plot_empirical_error_convergence(loss: npt.NDArray[np.float_], nn_config: dict, not_fixed_param: str, annotate: bool = False) -> None:
    sns.set(style="darkgrid", color_codes=True, rc={"figure.figsize": (8, 5)})
    ax = sns.lineplot(data=loss, markers=True, marker="o", label=f"{nn_config[not_fixed_param]}")

    del nn_config[not_fixed_param]
    keys = list(nn_config.keys())
    vls = list(nn_config.values())

    plt.xticks(np.arange(len(loss)))
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(title=not_fixed_param, fontsize=10)
    plt.title(f"Convergence of empirical error\n{keys[0]} = {vls[0]} , {keys[1]} = {vls[1]}")
    if annotate:
        for i, j in enumerate(loss):
            ax.annotate(str(np.round(j, 2)), xy=(i, j), xytext=(i, j + 0.02))

In [3]:
def plot_fixed(results: pd.DataFrame, hidden_layer_size: np.int_ = None, batch_size: np.int_ = None, learning_rate: np.float_ = None, annotate: bool = False) -> None:
    if hidden_layer_size is None and batch_size is None and learning_rate is None:
        raise "Especifique quais métricas devem ser fixadas"
    not_fixed: str = ""
    
    if hidden_layer_size and batch_size:
        aux: pd.DataFrame = results.query(f"hidden_layer_size == {hidden_layer_size} and batch_size == {batch_size}")[["learning_rate", "history"]]
        not_fixed = "learning_rate"
        for i in range(aux.shape[0]):
            e = aux.iloc[i]
            plot_empirical_error_convergence(loss=e.history["loss"], nn_config={"hidden_layer_size": hidden_layer_size, "batch_size": batch_size, "learning_rate": e.learning_rate}, not_fixed_param=not_fixed, annotate=annotate)
    
    if hidden_layer_size and learning_rate:
        aux: pd.DataFrame = results.query(f"hidden_layer_size == {hidden_layer_size} and learning_rate == {learning_rate}")[["batch_size", "history"]]
        not_fixed = "batch_size"
        for i in range(aux.shape[0]):
            e = aux.iloc[i]
            plot_empirical_error_convergence(loss=e.history["loss"], nn_config={"hidden_layer_size": hidden_layer_size, "batch_size": e.batch_size, "learning_rate": learning_rate}, not_fixed_param=not_fixed, annotate=annotate)
    
    if learning_rate and batch_size:
        aux: pd.DataFrame = results.query(f"batch_size == {batch_size} and learning_rate == {learning_rate}")[["hidden_layer_size", "history"]]
        not_fixed = "hidden_layer_size"
        for i in range(aux.shape[0]):
            e = aux.iloc[i]
            plot_empirical_error_convergence(loss=e.history["loss"], nn_config={"hidden_layer_size": e.hidden_layer_size, "batch_size": batch_size, "learning_rate": learning_rate}, not_fixed_param=not_fixed, annotate=annotate)

In [6]:
results: pd.DataFrame = pd.read_json("data/results.json")
results.nlargest(columns=["accuracy_score"], n=10)

,accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,random_state_seed,test_size,hidden_layer_size,batch_size,learning_rate,history
19,0.318667,"[0.276470588235294, 0.517857142857142, 0.16806...","[0.376, 0.533742331288343, 0.579710144927536, ...","[0.31864406779661003, 0.525679758308157, 0.260...","[[47, 1, 47, 5, 1, 4, 10, 3, 4, 3], [12, 87, 2...",42,0.3,50,50,1.0,"{'loss': [2.461884975433349, 2.349673509597778..."
28,0.315333,"[0.179012345679012, 0.5454545454545451, 0.3260...","[0.6960000000000001, 0.515337423312883, 0.2173...","[0.284779050736497, 0.529968454258675, 0.26086...","[[87, 1, 7, 11, 8, 0, 1, 4, 1, 5], [35, 84, 3,...",42,0.3,100,20,1.0,"{'loss': [2.934058904647827, 2.35768723487854,..."
3,0.314000,"[0.330882352941176, 0.5324675324675321, 0.2641...","[0.36, 0.503067484662576, 0.30434782608695604,...","[0.344827586206896, 0.5173501577287061, 0.2828...","[[45, 1, 20, 12, 1, 6, 13, 18, 4, 5], [10, 82,...",42,0.3,25,20,0.5,"{'loss': [2.333007335662842, 2.294318914413452..."
4,0.312000,"[0.32679738562091504, 0.521472392638036, 0.354...","[0.4, 0.521472392638036, 0.239130434782608, 0....","[0.35971223021582704, 0.521472392638036, 0.285...","[[50, 4, 8, 41, 0, 3, 1, 6, 8, 4], [9, 85, 1, ...",42,0.3,25,20,1.0,"{'loss': [2.372432947158813, 2.244149923324585..."
27,0.311333,"[0.30303030303030304, 0.655462184873949, 0.322...","[0.4, 0.47852760736196304, 0.21739130434782603...","[0.344827586206896, 0.553191489361702, 0.25974...","[[50, 1, 12, 12, 0, 19, 0, 3, 6, 22], [11, 78,...",42,0.3,100,20,0.5,"{'loss': [2.688820600509643, 2.393985748291015..."
16,0.307333,"[0.265765765765765, 0.663265306122448, 0.24752...","[0.47200000000000003, 0.398773006134969, 0.181...","[0.34005763688760804, 0.49808429118773906, 0.2...","[[59, 1, 10, 4, 33, 1, 3, 9, 1, 4], [18, 65, 1...",42,0.3,50,20,1.0,"{'loss': [2.504638433456421, 2.283196449279785..."
0,0.299333,"[0.48571428571428504, 0.46464646464646403, 0.2...","[0.136, 0.5644171779141101, 0.2173913043478260...","[0.21250000000000002, 0.509695290858725, 0.251...","[[17, 5, 9, 37, 9, 11, 4, 17, 13, 3], [0, 92, ...",42,0.3,25,1,0.5,"{'loss': [2.39508318901062, 2.168269157409668,..."
5,0.297333,"[0.49056603773584906, 0.777777777777777, 0.353...","[0.20800000000000002, 0.30061349693251505, 0.2...","[0.29213483146067404, 0.43362831858407, 0.2636...","[[26, 1, 11, 14, 38, 2, 1, 10, 16, 6], [1, 49,...",42,0.3,25,20,10.0,"{'loss': [2.663019895553589, 2.528800725936889..."
24,0.296000,"[0.413043478260869, 0.7560975609756091, 0.2529...","[0.152, 0.380368098159509, 0.31159420289855, 0...","[0.22222222222222202, 0.506122448979591, 0.279...","[[19, 1, 22, 9, 6, 29, 11, 6, 19, 3], [1, 62, ...",42,0.3,100,1,0.5,"{'loss': [2.517239332199096, 2.206258058547973..."
15,0.287333,"[0.226415094339622, 0.5642857142857141, 0.1552...","[0.48, 0.48466257668711604, 0.623188405797101,...","[0.30769230769230704, 0.521452145214521, 0.248...","[[60, 1, 47, 4, 1, 0, 5, 0, 0, 7], [16, 79, 39...",42,0.3,50,20,0.5,"{'loss': [2.398151397705078, 2.330489873886108..."
